In [1]:
%env PGINSTANCE=postgres
%env BASE_DIR=/home/jupyter/

env: PGINSTANCE=postgres
env: BASE_DIR=/home/jupyter/


In [2]:
!psql -c "\i $BASE_DIR/pagila/pagila-schema.sql" -q
!psql -c "\i $BASE_DIR/pagila/pagila-insert-data.sql" -q

psql:/home/jupyter/pagila/pagila-schema.sql:8: NOTICE:  drop cascades to 47 other objects
DETAIL:  drop cascades to type pagila.mpaa_rating
drop cascades to type pagila.year
drop cascades to function pagila._group_concat(text,text)
drop cascades to function pagila.film_in_stock(integer,integer)
drop cascades to function pagila.film_not_in_stock(integer,integer)
drop cascades to function pagila.get_customer_balance(integer,timestamp with time zone)
drop cascades to function pagila.inventory_held_by_customer(integer)
drop cascades to function pagila.inventory_in_stock(integer)
drop cascades to function pagila.last_day(timestamp with time zone)
drop cascades to function pagila.last_updated()
drop cascades to sequence pagila.customer_customer_id_seq
drop cascades to table pagila.customer
drop cascades to function pagila.rewards_report(integer,numeric)
drop cascades to function pagila.group_concat(text)
drop cascades to sequence pagila.actor_actor_id_seq
drop cascades to table pagila.actor


In [3]:
!psql -c "\dt pagila."
#!psql -c "SELECT * FROM pg_catalog.pg_tables WHERE schemaname = 'pagila'"

             List of relations
 Schema |     Name      | Type  |  Owner   
--------+---------------+-------+----------
 pagila | actor         | table | postgres
 pagila | address       | table | postgres
 pagila | category      | table | postgres
 pagila | city          | table | postgres
 pagila | country       | table | postgres
 pagila | customer      | table | postgres
 pagila | film          | table | postgres
 pagila | film_actor    | table | postgres
 pagila | film_category | table | postgres
 pagila | inventory     | table | postgres
 pagila | language      | table | postgres
 pagila | payment       | table | postgres
 pagila | rental        | table | postgres
 pagila | staff         | table | postgres
 pagila | store         | table | postgres
(15 rows)



In [4]:
# describe tables
# !psql -c "\d pagila."
!psql -c "\d pagila.actor"
!psql -c "\d pagila.address"
!psql -c "\d pagila.category"
!psql -c "\d pagila.city"
!psql -c "\d pagila.country"
!psql -c "\d pagila.customer"
!psql -c "\d pagila.film"
!psql -c "\d pagila.film_actor"
!psql -c "\d pagila.film_category"
!psql -c "\d pagila.inventory"
!psql -c "\d pagila.language"
!psql -c "\d pagila.payment"
!psql -c "\d pagila.rental"
!psql -c "\d pagila.staff"
!psql -c "\d pagila.store"

                                              Table "pagila.actor"
   Column    |           Type           | Collation | Nullable |                    Default                     
-------------+--------------------------+-----------+----------+------------------------------------------------
 actor_id    | integer                  |           | not null | nextval('pagila.actor_actor_id_seq'::regclass)
 first_name  | text                     |           | not null | 
 last_name   | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "actor_pkey" PRIMARY KEY, btree (actor_id)
    "idx_actor_last_name" btree (last_name)
Referenced by:
    TABLE "pagila.film_actor" CONSTRAINT "film_actor_actor_id_fkey" FOREIGN KEY (actor_id) REFERENCES pagila.actor(actor_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.actor FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()

     

In [5]:
# inner join 1
# returns the actor_id, first, and last names of actors starring in the film with id of 1, sorted by actor_id

!psql -c \
"select pa.actor_id, pa.first_name, pa.last_name \
from pagila.actor pa \
inner join pagila.film_actor pf \
on pa.actor_id = pf.actor_id \
where film_id = 1 \
order by actor_id;"


 actor_id | first_name | last_name 
----------+------------+-----------
        1 | PENELOPE   | GUINESS
       10 | CHRISTIAN  | GABLE
       20 | LUCILLE    | TRACY
       30 | SANDRA     | PECK
       40 | JOHNNY     | CAGE
       53 | MENA       | TEMPLE
      108 | WARREN     | NOLTE
      162 | OPRAH      | KILMER
      188 | ROCK       | DUKAKIS
      198 | MARY       | KEITEL
(10 rows)



In [6]:
# inner join 2
# returns the titles of all comedy movies, sorted by title

!psql -c \
"select pf.title \
from pagila.film pf \
inner join pagila.film_category pc \
on pf.film_id = pc.film_id \
inner join pagila.category p \
on pc.category_id = p.category_id \
where p.name = 'Comedy' \
order by title \
limit 10" 


         title          
------------------------
 AIRPLANE SIERRA
 ANTHEM LUKE
 BRINGING HYSTERICAL
 CAPER MOTIONS
 CAT CONEHEADS
 CLOSER BANG
 CONNECTION MICROCOSMOS
 CONTROL ANTHEM
 CRAZY HOME
 DADDY PITTSBURGH
(10 rows)



In [7]:
# inner join 3
# returns first and last name of customer whose amounts are greater than $5 and sorts results by earliest payment date

!psql -c \
'select pc.first_name, pc.last_name, pp.amount, pp.payment_date \
from pagila.customer pc \
inner join pagila.payment pp \
on pc.customer_id = pp.customer_id \
where amount > 5 \
order by payment_date \
limit 10;'


 first_name | last_name | amount |         payment_date          
------------+-----------+--------+-------------------------------
 DELORES    | HANSEN    |   6.99 | 2017-01-24 21:33:47.996577+00
 DANNY      | ISOM      |   5.99 | 2017-01-24 22:30:47.996577+00
 APRIL      | BURNS     |   8.99 | 2017-01-24 22:37:28.996577+00
 RAYMOND    | MCWHORTER |   6.99 | 2017-01-24 22:51:21.996577+00
 RONALD     | WEINER    |   9.99 | 2017-01-24 23:07:48.996577+00
 YVONNE     | WATKINS   |   5.99 | 2017-01-25 03:40:55.996577+00
 TROY       | QUIGLEY   |   7.99 | 2017-01-25 04:28:05.996577+00
 MARIA      | MILLER    |   5.99 | 2017-01-25 04:32:34.996577+00
 MANUEL     | MURRELL   |   5.99 | 2017-01-25 07:25:08.996577+00
 JO         | FOWLER    |   5.99 | 2017-01-25 07:30:23.996577+00
(10 rows)



In [8]:
# outer join 1
# return first and last name, and rental date for customers with a null return date, ordered by last name

!psql -c \
'select pc.first_name, pc.last_name, pr.rental_date, pr.return_date \
from pagila.customer pc \
full outer join pagila.rental pr \
on pc.customer_id = pr.customer_id \
where return_date is null \
order by last_name \
limit 10;'


 first_name | last_name |      rental_date       | return_date 
------------+-----------+------------------------+-------------
 RAFAEL     | ABNEY     | 2017-02-14 15:16:03+00 | 
 LISA       | ANDERSON  | 2017-02-14 15:16:03+00 | 
 JORDAN     | ARCHULETA | 2017-02-14 15:16:03+00 | 
 JORDAN     | ARCHULETA | 2017-02-14 15:16:03+00 | 
 MELANIE    | ARMSTRONG | 2017-02-14 15:16:03+00 | 
 DARRYL     | ASHCRAFT  | 2017-02-14 15:16:03+00 | 
 MILDRED    | BAILEY    | 2017-02-14 15:16:03+00 | 
 MILDRED    | BAILEY    | 2017-02-14 15:16:03+00 | 
 JESSIE     | BANKS     | 2017-02-14 15:16:03+00 | 
 DAISY      | BATES     | 2017-02-14 15:16:03+00 | 
(10 rows)



In [9]:
# outer join 2
# returns titles of films with no store_id, ordered by title

!psql -c \
'select pf.title \
from pagila.film pf \
full outer join pagila.inventory pi \
on pf.film_id = pi.film_id \
where store_id is null \
order by title \
limit 10;'


        title         
----------------------
 ALICE FANTASIA
 APOLLO TEEN
 ARGONAUTS TOWN
 ARK RIDGEMONT
 ARSENIC INDEPENDENCE
 BOONDOCK BALLROOM
 BUTCH PANTHER
 CATCH AMISTAD
 CHINATOWN GLADIATOR
 CHOCOLATE DUCK
(10 rows)



In [10]:
# outer join 3
# returns employee first name and last name, respective store id, for employees without a picture, sorted by store_id

!psql -c \
'select ps.first_name, ps.last_name, ps2.store_id \
from pagila.staff ps \
full outer join pagila.store ps2 \
on ps.store_id = ps2.store_id \
where picture is null \
order by store_id;'


 first_name | last_name | store_id 
------------+-----------+----------
 Jon        | Stephens  |        2
(1 row)



In [11]:
# self join
# returns list of first names and shared last name of actors with the same last name

!psql -c \
'select a.first_name actor_1_first_name, a2.first_name actor_2_first_name, a2.last_name \
from pagila.actor a \
inner join pagila.actor a2 \
on a.last_name = a2.last_name \
where a.first_name != a2.first_name \
limit 10;'


 actor_1_first_name | actor_2_first_name | last_name 
--------------------+--------------------+-----------
 CHRISTIAN          | KIRSTEN            | AKROYD
 CHRISTIAN          | DEBBIE             | AKROYD
 KIRSTEN            | CHRISTIAN          | AKROYD
 KIRSTEN            | DEBBIE             | AKROYD
 DEBBIE             | CHRISTIAN          | AKROYD
 DEBBIE             | KIRSTEN            | AKROYD
 CUBA               | KIM                | ALLEN
 CUBA               | MERYL              | ALLEN
 KIM                | CUBA               | ALLEN
 KIM                | MERYL              | ALLEN
(10 rows)

